## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-10-13-04-35 +0000,nyt,Six Takeaways From the Senate Deal to End the ...,https://www.nytimes.com/2025/11/10/us/politics...
1,2025-11-10-13-03-54 +0000,nyt,"DealBook Summit 2025: David Ellison, Scott Bes...",https://www.nytimes.com/2025/11/10/business/de...
2,2025-11-10-13-03-00 +0000,wsj,The decision by eight Democratic senators to s...,https://www.wsj.com/politics/policy/democrats-...
3,2025-11-10-13-00-26 +0000,nypost,Robert De Niro’s daughter shares emotional pos...,https://nypost.com/2025/11/10/us-news/robert-d...
4,2025-11-10-12-59-54 +0000,bbc,Seven dead after migrant boat sinks near Thai-...,https://www.bbc.com/news/articles/cd7rp5zj484o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
72,trump,36
5,shutdown,26
142,new,13
22,government,11
4,end,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
122,2025-11-10-05-02-42 +0000,nyt,Trump Renews Attacks on Obamacare in New Push ...,https://www.nytimes.com/2025/11/09/us/politics...,94
211,2025-11-09-18-46-21 +0000,nypost,SNAP thrown into chaos after Trump admin tells...,https://nypost.com/2025/11/09/us-news/snap-thr...,91
173,2025-11-09-22-28-52 +0000,nypost,Senate eyes vote on potential breakthrough to ...,https://nypost.com/2025/11/09/us-news/senate-e...,76
128,2025-11-10-04-14-40 +0000,nyt,Sean Duffy Juggles Shutdown Roles on Flight Sa...,https://www.nytimes.com/2025/11/09/us/politics...,74
2,2025-11-10-13-03-00 +0000,wsj,The decision by eight Democratic senators to s...,https://www.wsj.com/politics/policy/democrats-...,72


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
122,94,2025-11-10-05-02-42 +0000,nyt,Trump Renews Attacks on Obamacare in New Push ...,https://www.nytimes.com/2025/11/09/us/politics...
178,41,2025-11-09-22-01-33 +0000,wapo,Super Typhoon Fung-wong slams Philippines in w...,https://www.washingtonpost.com/world/2025/11/0...
173,39,2025-11-09-22-28-52 +0000,nypost,Senate eyes vote on potential breakthrough to ...,https://nypost.com/2025/11/09/us-news/senate-e...
159,36,2025-11-10-00-12-31 +0000,nypost,Boca Raton makes serious pitch to NYC business...,https://nypost.com/2025/11/09/us-news/boca-rat...
5,35,2025-11-10-12-59-13 +0000,nyt,"Nicolas Sarkozy, Former French President, Will...",https://www.nytimes.com/2025/11/10/world/europ...
37,32,2025-11-10-11-40-11 +0000,nypost,"Trump lashes out at ‘Corrupt,’ ‘dishonest’ BBC...",https://nypost.com/2025/11/10/us-news/trump-la...
161,32,2025-11-10-00-09-50 +0000,nypost,Brooklyn dad says goodbye to hero son Itay Che...,https://nypost.com/2025/11/09/world-news/nyc-d...
16,29,2025-11-10-12-30-00 +0000,wsj,Private-Label Food Maker TreeHouse to Go Priva...,https://www.wsj.com/business/retail/private-la...
2,25,2025-11-10-13-03-00 +0000,wsj,The decision by eight Democratic senators to s...,https://www.wsj.com/politics/policy/democrats-...
40,24,2025-11-10-11-19-12 +0000,cbc,"Trump pardons Rudy Giuliani, others who backed...",https://www.cbc.ca/news/world/rudy-giuliani-tr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
